In [1]:
from influxdb_client import InfluxDBClient
from dotenv import load_dotenv
import os
load_dotenv()

client = InfluxDBClient(
    url=os.getenv("INFLUXDB_URL"),
    token=os.getenv("INFLUXDB_TOKEN"),
    org=os.getenv("INFLUXDB_ORG"),
)
query_api = client.query_api()

In [2]:
from typing import Annotated
from typing_extensions import TypedDict

from langchain.agents import Tool
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list[str], add_messages]
    db_output: list
    next_inspection: list
    
graph_builder = StateGraph(State)

In [3]:
def query_tool():
    """InfluxDB에서 프로세스 ID에 대한 로그를 조회합니다."""
    query = f'''
    from(bucket: "P1-A_status")
    |> range(start: -1h)
    |> filter(fn: (r) => r._measurement == "status_log")
    '''
    try:
        tables = query_api.query(query)
        logs = []
        for table in tables:
            for record in table.records:
                logs.append(f"{record.get_time()}: {record.get_value()}")
        
        if not logs:
            # 검색 결과가 없는 경우
            if "-" in process_id:
                return f"라인 ID '{process_id}'에 대한 로그가 없습니다."
            else:
                return f"프로세스 ID '{process_id}'에 대한 로그가 없습니다."
        output = "\n".join(logs)
        return {"message": [output]}
    except Exception as e:
        return f"로그 조회 중 오류 발생: {e}"

In [4]:
from langgraph.prebuilt import ToolNode, tools_condition

tools = [query_tool]
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o")
llm_with_tools = llm.bind_tools(tools)

In [ ]:
GENERATE_SYSTEM_TEMPLATE = """
You are a database inspection assistant. You will be given a list of database queries and their results. Your task is to analyze the results and provide insights or suggestions for further actions."""
GENERATE_USER_TEMPLATE = """
You are a process monitoring assistant.
Your job is to analyze process logs and help diagnose issues.

When logs show errors, timeouts, or abnormal patterns, you can:
1. Request more information about specific processes
2. Send maintenance commands when necessary
use "query_tool" with input P1-A. """


def supervisor(state: State):
    msgs = [
        ("system", GENERATE_SYSTEM_TEMPLATE),
        ("user", GENERATE_USER_TEMPLATE)
    ]
    prompt = ChatPromptTemplate.from_messages(msgs)
    response = llm_with_tools.invoke(
        prompt.format_prompt(messages=state["messages"]),
    )
    outputs = []
    outputs.append(
        AIMessage(
            content=response.content,
        )
    )
    return {"messages": outputs}

graph_builder.add_node("supervisor", supervisor)

In [7]:
graph_builder.add_edge(START, "supervisor")
graph_builder.add_edge("supervisor", END)
graph_builder.add_conditional_edges("supervisor", tools_condition)
graph_builder.add_edge("tools", "supervisor")
graph = graph_builder.compile()

graph

ReadTimeout: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)

In [8]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}): # graph 노드 호출 결과 받아옴
        for value in event.values():
            print(value, "\n")

In [9]:
while True:
    try:
        user_input = "Use query_tool to get logs for process ID P1-A"
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        break

{'messages': [AIMessage(content='The command "use \'query_tool\' with input P1-A" indicates a specific request or action related to process P1-A. Before proceeding, let\'s analyze the context or details from your logs. Please provide the logs or any specific error messages, timeouts, or abnormal patterns you\'re observing. With this information, I can help identify the next steps or determine if further queries or maintenance actions are necessary.', additional_kwargs={}, response_metadata={}, id='612d906a-d53e-438c-86b3-e8d0da78d5ac')]} 

{'messages': [AIMessage(content='I\'m here to help analyze the logs for any errors, timeouts, or abnormal patterns. Let\'s proceed with the step you\'ve mentioned:\n\n1. I will start by reviewing the logs you\'ve provided using "query_tool" with input P1-A. \n2. I\'ll analyze the information for any irregularities and decide if additional information about specific processes is needed or if there\'s a need to send maintenance commands.\n\nPlease prov